In [23]:
import pandas as pd

from tqdm import tqdm
tqdm.pandas()

In [24]:
df = pd.read_csv('./atosLicitacao2.csv')

In [25]:
len(df)

20210

In [26]:
df.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 20210 entries, 0 to 20209
Data columns (total 5 columns):
 #   Column      Non-Null Count  Dtype 
---  ------      --------------  ----- 
 0   Unnamed: 0  20210 non-null  int64 
 1   id_ato      20210 non-null  int64 
 2   processo    15858 non-null  object
 3   data_dodf   20210 non-null  object
 4   texto       20210 non-null  object
dtypes: int64(2), object(3)
memory usage: 789.6+ KB


In [27]:
df.head()

,Unnamed: 0,id_ato,processo,data_dodf,texto
0,0,999,0000000000001970016272016,07-02-2019,EXTRATOS DE OUTORGA\nO SUPERINTENDENTE DE RECU...
1,1,14,0000000000000010006082018,07-02-2019,AVISO DE LICITACAO EXCLUSIVA\nPREGAO ELETRONIC...
2,2,999,NaN,07-02-2019,AVISO DE ABERTURA DO PLANO DE SUPRIMENTOS No 0...
3,3,999,NaN,07-02-2019,AVISO DO PREGAO ELETRONICO No 017/2019\nO BRB ...
4,4,999,0000000000000000010062018,07-02-2019,AVISO DO PREGAO ELETRONICO No 068/2018\nO BRB ...


In [28]:
import re

In [29]:
df['id_certame'] = range(1,len(df.processo.tolist())+1)

In [6]:
df_CERTAME = pd.DataFrame({'n_processo':df.processo.tolist(), 'id_certame': df.id_certame.tolist()})

df_CERTAME['insert'] = df_CERTAME.progress_apply(lambda row:'INSERT INTO CERTAME (id_certame, n_processo) VALUES ('+ str(row.id_certame) + ",'" + str(row.n_processo) + "');",axis=1)

100%|██████████| 20210/20210 [00:00<00:00, 24257.43it/s]


In [7]:
df_CERTAME.to_csv('./df_certame.csv')

In [61]:
import math
tipos_dict = {
    1: 'AVISO DE ABERTURA DE LICITACAO',
    2: 'AVISO DE HOMOLOGACAO E ADJUDICACAO',
    3: 'AVISO DE HOMOLOGACAO E CONVOCACAO',
    4: 'AVISO DE RESULTADO',
    5: 'AVISO DO RESULTADO DE JULGAMENTO',
    6: 'AVISO DE DECLARACAO DE VENCEDOR',
    7: 'AVISO DE JULGAMENTO',
    8: 'AVISO DE JULGAMENTO DE HABILITACAO',
    9: 'AVISO DE JULGAMENTO DE RECURSO ADMINISTRATIVO',
    10: 'AVISO DE SUSPENSAO DE LICITACAO',
    11: 'AVISO DE ADIAMENTO DE LICITACAO',
    12: 'AVISO DE NOVA DATA DE ABERTURA',
    13: 'AVISO DE REABERTURA',
    14: 'AVISO DE LICITACAO',
    15: 'EXTRATOS DE CONTRATO',
    999: 'DESCONHECIDO'
}

df['id_ato'] = df.apply(lambda row: 999 if math.isnan(float(row.id_ato) ) else row.id_ato, axis=1)
df['descricao'] = df.apply(lambda row: tipos_dict[row.id_ato], axis=1)

In [30]:
df.id_ato.value_counts()

999    13389
15      2716
4       1724
14      1397
3        325
1        223
2        173
13        93
7         76
10        41
6         27
12        19
11         7
Name: id_ato, dtype: int64

In [47]:
df_ATOS = pd.DataFrame({
    'id_ato': range(1,len(df.processo.tolist())+1),
    'data_dodf': df.data_dodf.tolist(),
    'texto': df.texto.tolist(),
    'id_certame': df.id_certame.tolist(),
    'id_tipo': df.id_ato.tolist()
})
df_ATOS['insert'] = df_ATOS.progress_apply(lambda row:'INSERT INTO ATOS (id_ato, data_dodf, texto, id_certame, id_tipo) VALUES ('+ str(row.id_ato) + "," + "TO_DATE('" + row.data_dodf + "','" + 'dd-MM-yyyy' + "')" + ",'" + re.sub("'|\"",' ',str(row.texto)) + "'," + str(row.id_certame) + ',' + str(row.id_tipo) + ');', axis=1)

100%|██████████| 20210/20210 [00:02<00:00, 8183.27it/s]


In [49]:
df_ATOS['insert'].to_csv('./df_atos.csv')

In [48]:
df_ATOS.head(1)['insert'].values[0]

"INSERT INTO ATOS (id_ato, data_dodf, texto, id_certame, id_tipo) VALUES (1,TO_DATE('07-02-2019','dd-MM-yyyy'),'EXTRATOS DE OUTORGA\nO SUPERINTENDENTE DE RECURSOS HIDRICOS DA AGENCIA REGULADORA DE AGUAS,\nENERGIA E SANEAMENTO BASICO DO DISTRITO FEDERAL - Adasa torna publicas as\no u t o rg a s :\nDespacho/SRH no 118/2018. VANDERLEI VIEIRA, concede outorga previa para perfuracao de um\npoco tubular, irrigacao e piscicultura, Bacia Hidrografica do Rio Descoberto, Nucleo Rural Alexandre\nGusmao, Gleba 03, Reserva F, Chacara F, Brazlandia/DF. Processo SEI no 0197-001627/2016.',1,999);"